In [3]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from squaternion import Quaternion

In [ ]:
class imuFilter:
    def __init__(self, R, Q):
        self.q = Quatenion()
        self.dt = 0.
        self.x = np.array([0]*10) # [qb,ab,wb]
        self.abias = 0
        self.wbias = 0
        self.qbias = Quaternion(0,0,0,0)
        self.R = R
        self.Q = Q
        self.P = Q
    
    def model(self, accel, gyro):
        """
        accel/gyro: (x,y,z) readings from IMU
        q: quaternion attitude estimate
        a: acceleration
        g: esitmate of gravity vector
        """
        abias = self.x[3:6]
        a = accel - self.abias
        wbias = self.x[6:]
        w = gyro - self.wbias
        q = self.q + 0.5*self.dt*self.q * Quaternion(0, w[0], w[1], w[2])
        g = np.array(q.to_rot())[2,:] # save a transpose g[2,:] = (g.T)[:,2]
        return q,a,g
        
    def correction(self):
        o = self.x[:4]
        ab = self.x[4:7]
        wb = self.x[7:]
        
        self.abias = self.abias*v-ab
        self.wbias = self.wbias-wb
        
    def kf(self,z,g):
        gx,gy,gz = g
        k = 0.001 # fixme
        self.x = np.array([0]*10)
        H = np.array([
            [  0,  gz, -gy,     0, -k*gz,  k*gy, 1, 0, 0],
            [-gz,   0,  gx,  k*gz,     0, -k*gx, 0, 1, 0],
            [ gy, -gx,   0, -k*gy,  k*gx,     0, 0, 0, 1]
        ])
        y = z
        s = self.R + H @ P @ H.T
        K = self.P @ H.T @ inv(s)        # kalman gain
        self.x = K @ y                   # a posteriori state
        self.P = self.P - K @ H @ self.P # a posteriori covariance
        
    def pose(self, accel, gyro):
        q,a,g = self.model(accel, gyro)
        z = a-g
        self.kf(z,g)
        self.correction()
        return (self.q, a, g)